In [ ]:
%config IPCompleter.greedy=True

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import pandas as pd

filename = './de_ch_vocabulary_spacy.csv'

word = 'word'
base = 'base'
names = [word, base]

voc = pd.read_csv(filename, sep=',', encoding='ansi', comment='#', names=names)
len(voc)

### Import Words from Ionesoft with grammar errors
Ich nehme an ein paar davon haben doppel s anstatt ß und wurden darum nicht gefunden.

In [ ]:
import pickle

missing_words = pickle.load(open('missing_words_grammar.p', 'rb'))
missing_words[:5]

#### Rechtschreibefehler Lösungsversuch

In [ ]:
import itertools
import Levenshtein

def similarity(w1, w2):
    return Levenshtein.jaro(w1.lower(), w2.lower())

def get_closest_match(wrong_word):
    voc['sim'] = voc[word].apply(lambda w: similarity(str(w), wrong_word))
    return voc.iloc[voc['sim'].idxmax()]['base']

In [ ]:
get_closest_match('apgemeldet')

In [ ]:
error_correct_dict = dict.fromkeys(missing_words)

In [ ]:
for w in missing_words:
    error_correct_dict[w] = get_closest_match(w)

In [ ]:
import csv

with open('ionesoft_grammar_corrected.csv', 'w') as f:
    for key in error_correct_dict.keys():
        f.write("%s,%s\n"%(key, error_correct_dict[key]))

#### manuel überprüfen und korrigieren
Konnte 570 Wörter manuell korrigieren und überprüfen

#### korrigiertes csv importieren

In [166]:
import csv

with open('ionesoft_grammar_manual_correction.csv', mode='r', ) as f:
    corrected_dict = dict(filter(None, csv.reader(f, delimiter=';')))

len(corrected_dict) - 1

570

In [167]:
import pandas as pd

filename = './ionesoft_tickets.csv'

tickets = pd.read_csv(filename, sep=',', encoding='ansi')
tickets.head()

,ticket,type,client,issuer,inquiry,answer
0,2.019031e+15,Fehler in der App,SVBA,3,ich kann die bilder in übungen nicht bearbeite...,Fehlerbericht senden
1,2.019030e+15,Fehler in der App,FOMA,3,lückentexte weg daten verloren gegangen von pp...,Fehlerbericht senden
2,2.019023e+15,Anmeldung / Aktivierung,AGVS,3,bitte deaktivieren sie dieses gerät. vielen dank.,"Gerät entfernt, neu starten"
3,2.019023e+15,Geräteanzahl überschritten,BEOOK,3,ich habe einen neuen laptop und möchte den alt...,"Gerät entfernt, neu starten"
4,2.019023e+15,Geräteanzahl überschritten,BEOOK,3,offenbar habe ich die geräteanzahl überschritt...,"Gerät entfernt, neu starten"


In [168]:
len(corrected_dict)

571

#### find wrongly written words

In [169]:
occurances = []

for target in corrected_dict.keys():
    o = tickets[tickets['inquiry'].str.contains(target, regex=False)]['inquiry']
    if len(o) > 0:
        occurances.append(o)
    
len(occurances)

478

#### replace them

In [170]:
for k in corrected_dict.keys():
    tickets['inquiry'] = tickets['inquiry'].str.replace(k, corrected_dict[k], regex=False)

#### check head

In [171]:
tickets.head()

,ticket,type,client,issuer,inquiry,answer
0,2.019031e+15,Fehler in der App,SVBA,3,ich kann die bilder in übungen nicht bearbeite...,Fehlerbericht senden
1,2.019030e+15,Fehler in der App,FOMA,3,lückentexte weg daten verloren gegangen von pp...,Fehlerbericht senden
2,2.019023e+15,Anmeldung / Aktivierung,AGVS,3,bitte deaaktivieren sie dieses gerät. vielen d...,"Gerät entfernt, neu starten"
3,2.019023e+15,Geräteanzahl überschritten,BEOOK,3,ich habe einen neuen laptop und möchte den alt...,"Gerät entfernt, neu starten"
4,2.019023e+15,Geräteanzahl überschritten,BEOOK,3,offenbar habe ich die gerägeräteanzahl übersch...,"Gerät entfernt, neu starten"


#### check replaced words

In [172]:
occurances = []

for target in corrected_dict.keys():
    space_or_point_wrapped_or_komma = ' ' + target + ' ' + '|' + ' ' + target + '.' + '|' + ' ' + target + ','
    o = tickets[tickets['inquiry'].str.contains(space_or_point_wrapped, regex=True)]['inquiry']
    if len(o) > 0:
        print(target)
        occurances.append(o)
    
len(occurances)

0

In [174]:
tickets.to_csv (r'./ionesoft_tickets_corrected_grammar.csv', index = None, header=True)

From **571** wrongly written words I found **478** words. I could correct **478** words